In [1]:
#!pip install h2o

In [2]:
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np

import pylab as plt
import seaborn as sns

%matplotlib inline

pd.set_option('display.max_columns', None)
from sklearn.metrics import mean_squared_error as mse

In [3]:
rbnb = pd.read_csv('../data/train.csv')
rbnb_test = pd.read_csv('../data/test.csv')

# H2O

## Entreno el modelo

In [4]:

from sklearn.model_selection import train_test_split as tts


train, test = tts(rbnb, test_size=.2, random_state=42)

train.shape, test.shape

((3333, 74), (834, 74))

In [5]:
import h2o
from h2o.automl import H2OAutoML

In [6]:
# inicializamos el modelo h2o

h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321 ..... not found.
Attempting to start a local H2O server...
; Java HotSpot(TM) 64-Bit Server VM (build 18.0.1.1+2-6, mixed mode, sharing)
  Starting server from C:\Users\usuario\.conda\envs\clase\Lib\site-packages\h2o\backend\bin\h2o.jar
  Ice root: C:\Users\usuario\AppData\Local\Temp\tmpw71af839
  JVM stdout: C:\Users\usuario\AppData\Local\Temp\tmpw71af839\h2o_usuario_started_from_python.out
  JVM stderr: C:\Users\usuario\AppData\Local\Temp\tmpw71af839\h2o_usuario_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O_cluster_uptime:,01 secs
H2O_cluster_timezone:,Europe/Madrid
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.36.1.1
H2O_cluster_version_age:,25 days
H2O_cluster_name:,H2O_from_python_usuario_e7m3js
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,2.924 Gb
H2O_cluster_total_cores:,8
H2O_cluster_allowed_cores:,8
H2O_cluster_status:,"locked, healthy"


In [7]:
h2train=h2o.H2OFrame(train)
h2test=h2o.H2OFrame(test)

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


In [8]:
X=[c for c in h2train.columns if c!='price']

y='price'

In [9]:
# inicia auto-machine-learning

automl=H2OAutoML(max_models=50,
                 seed=42,   # random_state
                 max_runtime_secs=300,
                 sort_metric='RMSE')

In [10]:
# entrena

automl.train(x=X,
             y=y,
             training_frame=h2train)

AutoML progress: |
19:48:14.143: AutoML: XGBoost is not available; skipping it.
19:48:14.174: _train param, Dropping bad and constant columns: [amenities, picture_url, neighbourhood_group_cleansed, calendar_updated, listing_url, name, description, bathrooms, scrape_id]

██████
19:48:37.716: GLM_1_AutoML_1_20220508_194814 [GLM def_1] failed: DistributedException from /127.0.0.1:54321: 'Java heap space', caused by java.lang.OutOfMemoryError: Java heap space
19:48:38.643: _train param, Dropping bad and constant columns: [amenities, picture_url, neighbourhood_group_cleansed, calendar_updated, listing_url, name, description, bathrooms, scrape_id]

█
19:48:43.914: _train param, Dropping bad and constant columns: [amenities, picture_url, neighbourhood_group_cleansed, calendar_updated, listing_url, name, description, bathrooms, scrape_id]

██████████
19:49:30.231: _train param, Dropping bad and constant columns: [amenities, picture_url, neighbourhood_group_cleansed, calendar_updated, listing_u

,,number_of_trees,number_of_internal_trees,model_size_in_bytes,min_depth,max_depth,mean_depth,min_leaves,max_leaves,mean_leaves
0,,62.0,62.0,60913.0,6.0,6.0,6.0,25.0,63.0,46.935482




ModelMetricsRegression: gbm
** Reported on train data. **

MSE: 1129.636224245388
RMSE: 33.61006135438297
MAE: 22.851439940975908
RMSLE: 0.24938959792540794
Mean Residual Deviance: 1129.636224245388

ModelMetricsRegression: gbm
** Reported on cross-validation data. **

MSE: 9481.229312619731
RMSE: 97.37160424178977
MAE: 50.34154293943399
RMSLE: NaN
Mean Residual Deviance: 9481.229312619731

Cross-Validation Metrics Summary: 


,,mean,sd,cv_1_valid,cv_2_valid,cv_3_valid,cv_4_valid,cv_5_valid
0,mae,49.971226,1.505475,50.103844,50.446774,50.132267,47.525032,51.648212
1,mean_residual_deviance,9599.678000,1724.088000,8462.748000,8648.253000,11215.213000,7969.221000,11702.951000
2,mse,9599.678000,1724.088000,8462.748000,8648.253000,11215.213000,7969.221000,11702.951000
3,r2,0.495507,0.380268,0.880405,0.344296,0.019826,0.892049,0.340958
4,residual_deviance,9599.678000,1724.088000,8462.748000,8648.253000,11215.213000,7969.221000,11702.951000
5,rmse,97.668350,8.701325,91.993195,92.995990,105.901900,89.270490,108.180180
6,rmsle,0.405098,0.030909,NaN,0.375265,0.391444,0.406295,0.447386



Scoring History: 


,,timestamp,duration,number_of_trees,training_rmse,training_mae,training_deviance
0,,2022-05-08 19:51:24,4.049 sec,0.0,193.467206,80.227498,37429.559685
1,,2022-05-08 19:51:24,4.118 sec,5.0,135.433738,61.924184,18342.297273
2,,2022-05-08 19:51:24,4.172 sec,10.0,97.307366,50.877723,9468.723416
3,,2022-05-08 19:51:24,4.237 sec,15.0,77.756957,44.424621,6046.144424
4,,2022-05-08 19:51:24,4.303 sec,20.0,63.489930,39.715187,4030.971234
5,,2022-05-08 19:51:24,4.372 sec,25.0,54.165336,36.185145,2933.883653
6,,2022-05-08 19:51:24,4.447 sec,30.0,49.153908,33.301696,2416.106656
7,,2022-05-08 19:51:24,4.521 sec,35.0,45.490872,30.776134,2069.419426
8,,2022-05-08 19:51:24,4.615 sec,40.0,42.356263,28.780631,1794.052977
9,,2022-05-08 19:51:24,4.717 sec,45.0,39.862837,27.046024,1589.045788



Variable Importances: 


,variable,relative_importance,scaled_importance,percentage
0,host_neighbourhood,114980112.0,1.000000,0.197864
1,property_type,106546952.0,0.926655,0.183352
2,calculated_host_listings_count_private_rooms,53919128.0,0.468943,0.092787
3,neighbourhood_cleansed,41548060.0,0.361350,0.071498
4,accommodates,38347488.0,0.333514,0.065991
5,host_verifications,30356480.0,0.264015,0.052239
6,license,21622782.0,0.188057,0.037210
7,bedrooms,20475400.0,0.178078,0.035235
8,host_since,20156300.0,0.175302,0.034686
9,host_acceptance_rate,14325252.0,0.124589,0.024652



See the whole table with table.as_data_frame()


In [11]:
print('[INFO] Leader board:')

leader_board=automl.leaderboard

leader_board.head()

[INFO] Leader board:


model_id,rmse,mse,mae,rmsle,mean_residual_deviance
GBM_5_AutoML_1_20220508_194814,97.3716,9481.23,50.3415,nan,9481.23
GBM_grid_1_AutoML_1_20220508_194814_model_5,100.865,10173.8,52.8783,nan,10173.8
GBM_2_AutoML_1_20220508_194814,100.893,10179.3,54.5709,nan,10179.3
GBM_3_AutoML_1_20220508_194814,101.091,10219.3,54.5379,nan,10219.3
GBM_4_AutoML_1_20220508_194814,101.222,10245.8,53.4855,nan,10245.8
GBM_grid_1_AutoML_1_20220508_194814_model_2,102.36,10477.7,55.4282,nan,10477.7
GBM_grid_1_AutoML_1_20220508_194814_model_1,102.703,10548,53.5857,nan,10548
GBM_grid_1_AutoML_1_20220508_194814_model_4,104.065,10829.6,55.2224,nan,10829.6
GBM_grid_1_AutoML_1_20220508_194814_model_3,105.491,11128.3,53.9412,nan,11128.3
DRF_1_AutoML_1_20220508_194814,116.561,13586.4,51.8256,0.419024,13586.4


In [12]:
H20_1 = automl.leader

In [13]:
# prediciones del lider

y_pred=automl.leader.predict(h2test)

gbm prediction progress: |███████████████████████████████████████████████████████| (done) 100%


In [14]:
# Para testear la precisión de mi modelo

'''from sklearn.metrics import accuracy_score

accuracy_score(y_test, y_pred_test)'''

'from sklearn.metrics import accuracy_score\n\naccuracy_score(y_test, y_pred_test)'

## Prediccion del test real para hacer el summit

In [15]:
h2_real_test=h2o.H2OFrame(rbnb_test)

y_pred_test = automl.leader.predict(h2_real_test)

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
gbm prediction progress: |███████████████████████████████████████████████████████| (done) 100%


In [16]:
submission = h2o.as_list(y_pred_test)

In [17]:
rbnb_test_y = rbnb_test.id

In [18]:
submission['id'] = rbnb_test_y

In [19]:
submission = submission.rename(columns={'predict': 'price'})

In [20]:
submission.to_csv('../data/H2O.csv', sep=',', index=False)

In [21]:
y_pred = h2o.as_list(y_pred)



In [22]:
mse(test.price, y_pred.predict, squared=False)

94.42480407277844